In [5]:
from langchain.agents import AgentType, initialize_agent, Tool, AgentExecutor
from langchain.chains import LLMChain
from langchain_community.llms import GigaChat
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv
import os
import PyPDF2
from pypdf import PdfReader

In [1]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.7 MB/s eta 0:00:00


In [2]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.5/305.5 kB 5.7 MB/s eta 0:00:00


In [3]:
!pip install gigachat

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 2.0 MB/s eta 0:00:00


In [4]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.1 MB/s eta 0:00:00


In [7]:
load_dotenv()

True

In [47]:
class HRAgent:
    def __init__(self):
        self.llm = GigaChat(
            credentials=os.getenv('GIGACHAT_API_PERS'),
            verify_ssl_certs=False,
            temperature=0.3,
            top_p=0.8
        )
        self.memory = ConversationBufferMemory(memory_key='chat_history')
        self.tools = self._setup_tools()
        self.agent = initialize_agent(
            self.tools,
            self.llm,
            agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
            memory=self.memory,
            verbose=True,
            handle_parsing_errors=True,
            max_iterations=1
        )

    def _setup_tools(self):
        return [
            Tool(
                name='analyze_resume',
                func=self._analyze_resume,
                description='Анализирует соответствие резюме вакансии. input: текст вакансии и текст резюме. output: оценка соответствия и обоснование.'
            ),
            Tool(
                name='generate_job_description',
                func=self._generate_job_description,
                description='Генерирует или улучшает описание вакансии. input: должность, обязанности, требования. output: текст вакансии.'
            ),
            Tool(
                name='generate_interview_questions',
                func=self._generate_interview_questions,
                description='Генерирует вопросы для собеседования. input: описание вакансии. output: список вопросов.'
            )
        ]

    def _extract_pdf_text(self, file_path):
        '''функция для извлечения текста из pdf'''
        with open(file_path, 'rb') as file:
            reader = PdfReader(file)
            text = "\n".join([page.extract_text() for page in reader.pages])
        return text

    def _analyze_resume(self, vacancy_pdf: str, resume_pdf: str) -> dict:
        '''анализ соответствия резюме вакансии'''
        vacancy_text = self._extract_pdf_text(vacancy_pdf)
        resume_text = self._extract_pdf_text(resume_pdf)

        template = '''Проанализируй соответствие резюме вакансии и дай оценку:

        Вакансия:
        {vacancy}

        Резюме:
        {resume}

        Оцени соответствие по шкале: Подходит / Частично подходит / Не подходит.
        Дай развернутое обоснование, сравнивая ключевые требования вакансии с опытом, образованием и навыками кандидата.
        Укажи сильные и слабые стороны кандидата относительно данной позиции.'''

        prompt = PromptTemplate(
        input_variables=['vacancy', 'resume'],
        template=template
        )

        chain = LLMChain(llm=self.llm, prompt=prompt)
        return chain.run({
            'vacancy': vacancy_text.strip(),
            'resume': resume_text.strip()
            })

    def _generate_job_description(self, inputs):
        '''генерация описания вакансии'''
        position, responsibilities, requirements = inputs.split('|||')

        template = '''Создай профессиональное описание вакансии на основе следующих данных:

        Должность: {position}
        Обязанности: {responsibilities}
        Требования: {requirements}

        Сделай текст:
        - Четким и понятным
        - С правильной структурой (о компании, обязанности, требования, условия)
        - С профессиональной лексикой'''

        prompt = PromptTemplate(
            input_variables=['position', 'responsibilities', 'requirements'],
            template=template
        )

        chain = LLMChain(llm=self.llm, prompt=prompt)
        return chain.run({
            'position': position,
            'responsibilities': responsibilities,
            'requirements': requirements
        })

    def _generate_interview_questions(self, vacancy_text):
        '''шенерация вопросов на собеседование по определённой вакансии'''
        template = '''На основе описания вакансии:

        Вакансия:
        {vacancy}

        сгенерируй вопросы для собеседования на такие темы, как:
        1. Технические вопросы (hard skills)
        2. Поведенческие вопросы (soft skills)
        3. Ситуационные вопросы
        4. Вопросы о мотивации

        Для каждого вопроса напиши обоснование, почему он задан.'''

        prompt = PromptTemplate(
            input_variables=['vacancy'],
            template=template
        )

        chain = LLMChain(llm=self.llm, prompt=prompt)
        return chain.run({'vacancy': vacancy_text})

    def run(self, task_description):
        return self.agent.run(task_description)

## 1. Проанализируем соответствие резюме вакансии

In [61]:
agent = HRAgent()

In [63]:
result = agent._analyze_resume(
    vacancy_pdf='Стажёр-инженер ML.pdf',
    resume_pdf='Соломенцева Арина ML-инженер.pdf'
)
print(result)

### Оценка соответствия резюме вакансии

#### Ключевые требования вакансии:
1. **Формирование рекомендательной ленты** – включает расчет признаков, обучение и применение ML/DL моделей, таргеты и формулы ранжирования, предсказание рекламной выручки, правила разнообразия.
2. **Full stack Data Scientist** – от проверки идей на Spark и Python до разработки production-ready систем на Go и А/Б тестирования.
3. **Стек технологий**: Python, PyTorch, Golang, Hadoop stack (pySpark, hive, hdfs), Catboost, Airflow.
4. **Задачи**: Обучение рекомендательных нейросетей, эксперименты с архитектурой нейросети, оптимизация пайплайнов обучения/сборки данных, написание runtime движка на Golang.

#### Соответствие резюме:

1. **Опыт работы**:
   - **Яндекс Крауд** (1 год 2 месяца): Специалист по разметке данных. В этой роли кандидат занимался разметкой данных для обучения нейросетей (изображения, аудио, видео, текстовые данные), оценкой и сравнением ответов LLM, разметкой данных для CV-проектов, оценкой ре

## 2. Генерация описания вакансии

In [11]:
position = 'Data Scientist'
responsibilities = '''
- Разработка и внедрение ML-моделей
- Анализ больших данных
- Визуализация результатов
- Коллаборация с продуктовой командой
'''
requirements = '''
- Опыт работы с Python, PyTorch/TensorFlow
- Знание SQL и NoSQL баз данных
- Понимание методов машинного обучения
- Опыт работы от 3 лет
'''

In [17]:
result = agent.run(f'''
Сгенерируй описание вакансии для:
Должность: {position} |||
Обязанности: {responsibilities} |||
Требования: {requirements}
Используй инструмент generate_job_description
''')
print(result)



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: generate_job_description
Action Input: Должность: Data Scientist ||| Обязанности: 
- Разработка и внедрение ML-моделей
- Анализ больших данных
- Визуализация результатов
- Коллаборация с продуктовой командой
 ||| Требования: 
- Опыт работы с Python, PyTorch/TensorFlow
- Знание SQL и NoSQL баз данных
- Понимание методов машинного обучения
- Опыт работы от 3 лет

Observation: Output:
Data Scientist
Разработка и внедрение ML-моделей
Анализ больших данных
Визуализация результатов
Коллаборация с продуктовой командой

Требуемые навыки:
- Опыт работы с Python, PyTorch/TensorFlow
- Знание SQL и NoSQL баз данных
- Понимание методов машинного обучения
- Опыт работы от 3 лет
Observation: **Вакансия: Data Scientist**

**Обязанности:**
- Разработка и внедрение передовых моделей машинного обучения для решения бизнес-задач.
- Проведение глубокого анализа больших объемов данных для выявления скрытых закономерностей и

## 3. Генерация вопросов для собеседования

In [36]:
vacancy_text = agent._extract_pdf_text('Стажёр-инженер ML.pdf')

In [37]:
result = agent.run(f'''
Сгенерируй 8 вопросов для собеседования на основе этой вакансии:
{vacancy_text}
Используй инструмент generate_interview_questions
''')
print(result)



> Entering new AgentExecutor chain...
Could not parse LLM output: `### Инструмент: generate_interview_questions

**Входные данные:** Вакансия: стажер-инженер ML, Должность: Стажер-инженер ML, Обязанности: обучение рекомендательных нейросетей, проведение экспериментов с архитектурой нейросети, оптимизация пайплайнов обучения/сборки данных, написание runtime двигателя на Golang. Требования: хорошее владение Python, учебный опыт программирования на Go/C++, учебный или коммерческий опыт работы с нейросетями, хороший уровень владения SQL, понимание теоретических принципов в основе классического ML и Deep Learning.

**Выходные данные:** Восемь вопросов для собеседования на позицию стажера-инженера ML.

---

### Вопросы для собеседования:

1. **Опишите ваш опыт работы с нейросетями и их обучением.**
2. **Какие методы оптимизации вы применяли при работе с пайплайнами обучения/сборки данных?**
3. **Как бы вы описали свой опыт работы с PyTorch и его применение в реальных проектах?**
4. **Что в